In [ ]:
import numpy as np
import os
from matplotlib import image
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import cv2 as cv

from terrain_nerf.feature_map import FeatureMap, px_to_global, depth_to_global

# autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
# Global image

UNREAL_PLAYER_START = np.array([-117252.054688, 264463.03125, 25148.908203])
UNREAL_GOAL = np.array([-83250.0, 258070.0, 24860.0])

global_img = image.imread('../data/airsim/images/test_scenario.png')
fig = px.imshow(global_img[:, :, :3])
fig.update_layout(width=1600, height=800)
fig.show()

In [ ]:
# Create feature map
start_px = (271, 257)
goal_px = (170, 808)

fm = FeatureMap(global_img, start_px, goal_px, UNREAL_PLAYER_START, UNREAL_GOAL)

In [ ]:
# Rover image taken from start location
rover_img = image.imread('../data/airsim/images/front_1688104627.4594698.png')
depth = np.load('../data/airsim/images/depth_1688104615.1395848.npy')

depth_thresh = 50  # meters
depth_img = depth.copy()
depth_img[depth > depth_thresh] = depth_thresh

# Show rover image and depth image side by side
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(rover_img)
ax[1].imshow(depth_img)
plt.show()

In [ ]:
cam_params = {'w': 800,
              'h': 600,
              'cx': 400, 
              'cy': 300, 
              'fx': 400, 
              'fy': 300}

In [ ]:
from terrain_nerf.utils import euler_to_R

R = euler_to_R(0, 0, np.radians(-30))
R = np.eye(3)

In [ ]:
cam_pose = np.hstack((R, np.zeros((3,1))))

In [ ]:
px_to_global(depth, cam_pose, cam_params, (300, 400))

In [ ]:
# Plot global image with local points overlaid
fig, ax = plt.subplots(1, 1, figsize=(20, 10))
ax.imshow(global_img, extent=fm.bounds)

# For each pixel in rover image, get feature from global image
G = depth_to_global(depth, cam_pose, cam_params, depth_thresh=100, patch_size=10)
ax.scatter(G[:,0], G[:,1], c='r', s=1)

plt.show()